In [ ]:
import os
import json
import pickle
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [ ]:
entities_file = r"C:\Users\fabien\Documents\Arvind\GreaseLM\utils_biomed\entities.jsonl"
relations_file = r"C:\Users\fabien\Documents\Arvind\GreaseLM\utils_biomed\relations.jsonl"
save_entities_file = r"C:\Users\fabien\Documents\Arvind\GreaseLM\data\ddb\ddb_names.json"
save_relations_file = r"C:\Users\fabien\Documents\Arvind\GreaseLM\data\ddb\ddb_relas.json"
save_ddb_to_umls_cui_file = r"C:\Users\fabien\Documents\Arvind\GreaseLM\data\ddb\ddb_to_umls_cui.txt"

In [ ]:
with open(entities_file) as f:
    lines = f.readlines()
    db_parsed_entities = [json.loads(line) for line in lines]

with open(relations_file) as f:
    lines = f.readlines()
    db_parsed_relations = [json.loads(line) for line in lines]

In [ ]:
escapes = ''.join([chr(char) for char in range(1, 32)])
trantab = str.maketrans("","", escapes)

In [ ]:
db_entities_json = {}
db_to_umls = set()
umls_to_db =  dict()
db_entity_ids = defaultdict(lambda: len(db_entity_ids))

In [ ]:
for parsed_entities in db_parsed_entities:
    entities = parsed_entities["entities"]
    for entity in entities:
        name = entity["text"]
        name = name.translate(trantab)
        linking_results = entity["linking_results"]
        if linking_results:
            for result in linking_results:
                preferred_name = result["Canonical Name"]
                if preferred_name == 'No Information Available':
                    preferred_name = name+"-"
                cui = result["Concept ID"]
                db_id = db_entity_ids[cui]
                db_to_umls.add((str(db_id), cui))
                umls_to_db[cui] = str(db_id)
                #if db_id == 119:
                #print(result)
                """
                if name not in db_entities_json:
                    if name == preferred_name:
                        is_preferred_name = "1"
                        db_entities_json[name] = [str(db_id), is_preferred_name]
                    else:
                        is_preferred_name = "0"
                        db_entities_json[name] = [str(db_id), is_preferred_name]
                """
                # only incorporate preferred names - we just need to the nodes/edges anyway
                if preferred_name not in db_entities_json:
                    is_preferred_name = "1"
                    db_entities_json[preferred_name] = [str(db_id), is_preferred_name]
        else:
            preferred_name = name
            cui = name
            db_id = db_entity_ids[cui]
            db_to_umls.add((str(db_id), cui))
            umls_to_db[cui] = str(db_id)
            if name not in db_entities_json:
                is_preferred_name = "1"
                db_entities_json[name] = [str(db_id), is_preferred_name]

In [ ]:
db_relations_json = {}
db_relation_ids = defaultdict(lambda: len(db_relation_ids))

In [ ]:
for relation in db_parsed_relations:
    subj = relation["head"]
    subj = subj.translate(trantab)
    obj = relation["tail"]
    obj = obj.translate(trantab)
    rel = relation["type"]
    rel = rel.translate(trantab)
    
    if subj in umls_to_db:
        subj = umls_to_db[subj]
    if obj in umls_to_db:
        obj = umls_to_db[obj]
    
    add_relation = (subj, obj, rel)
    relation_id = db_relation_ids[add_relation]
    
    if relation_id not in db_relations_json:
        db_relations_json[relation_id] = list(add_relation)

In [ ]:
sorted(list(db_to_umls))

In [ ]:
"C1512018" in umls_to_db, "C0018517" in umls_to_db, ("49", 'C0449379') in db_to_umls, db_entities_json['information-'], db_entities_json['Kidney'] 

In [ ]:
for k,v in db_entities_json.items():
    #if v[0] == "49":
    #if v[0] == "58":
    if v[0] == "119":
        print(k)
        print(v)

In [ ]:
"""
for parsed_entities in db_parsed_entities:
    entities = parsed_entities['entities']
    for entity in entities:
        #print("entity: {}".format(entity))
        if 'linking_results' in entity:
            linking_results = entity['linking_results']
            #print("linking results: {}".format(linking_results))
            for res in linking_results:
                #print("CUI: {}".format(res['Concept ID']))
                if "C1512018" in res['Concept ID']:
                    print(entity)
                    print("CUI: {}".format(res['Concept ID']))
                #if "C" in res['Concept ID']:
                #print("CUI: {}".format(res['Concept ID']))
"""

In [ ]:
with open(save_ddb_to_umls_cui_file, 'w', encoding='utf-8') as f:
    f.write('\t'.join(["LinkItemsToUMLSCUIID", "ItemPTR", "CUI", "ItemToUMLSCUILinkTypePTR"]) + '\n')
    for row in sorted(list(db_to_umls)):
        db_ptr = row[0]
        cui = row[1]
        row = ["0", str(db_ptr), cui, "0"]
        f.write('\t'.join(row) + '\n')

In [ ]:
with open(save_entities_file, 'w') as f:
    json.dump(db_entities_json, f)

In [ ]:
with open(save_relations_file, 'w') as f:
    json.dump(db_relations_json, f)